# 1. Overview
In this lab, we will show you how to create a multimodal (text + images) vector index in **Azure AI Search.**

### Prerequisites
- 🐍 Python 3.9 or higher
- ☁️ [Azure Blob storage](https://learn.microsoft.com/azure/storage/common/storage-account-create), used as the data source during indexing.
- 🔗 Azure AI Vision Service or Azure AI Multi-Service Account (https://learn.microsoft.com/azure/ai-services/multi-service-resource), used for multi-modal embeddings.
- 🔗 [Azure AI Search](https://learn.microsoft.com/azure/search/search-create-service-portal), any region and tier, but we recommend Basic or higher for this workload.

We use the [Azure Python SDK](https://learn.microsoft.com/en-us/python/api/azure-search-documents/?view=azure-python-preview) for indexer-driven indexing and vector query operations.

For indexing, the pattern uses the built in **Vision Vectorizer skill** to call the Image Retrieval API. Provisioning of this search service, **AI Services** account, and setup of the indexer is fully automated and included as a step in this notebook.

The AI services accounts is also used during queries, as the vectorizer. A vectorizer specifies which embedding model to use for vectorizing a text query string or an images. As always, it's strongly recommended that query vectorization is performed using the same embedding model used for document vectorization during indexing.


### Import libraries

In [ ]:
import os

from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.indexes.models import (
    AIServicesVisionParameters,
    AIServicesVisionVectorizer,
    BlobIndexerImageAction,
    CognitiveServicesAccountKey,
    FieldMapping,
    HnswAlgorithmConfiguration,
    HnswParameters,
    ImageAnalysisSkill,
    IndexerExecutionStatus,
    IndexingParameters,
    IndexingParametersConfiguration,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    ScalarQuantizationCompressionConfiguration,
    ScalarQuantizationParameters,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection,
    SearchIndexerSkillset,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
    VisionVectorizeSkill,
    MergeSkill
)

from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
from IPython.display import Image, display, HTML
from openai import AzureOpenAI

### Read environment variables

In [ ]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv(override=True)

# Configuration
AZURE_AI_VISION_API_KEY = os.getenv("AZURE_AI_SERVICES_API_KEY")
AZURE_AI_VISION_ENDPOINT = os.getenv("AZURE_AI_SERVICES_ENDPOINT")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
BLOB_CONNECTION_STRING = os.getenv("BLOB_CONNECTION_STRING")
BLOB_CONTAINER_NAME = os.getenv("AZURE_STORAGE_CONTAINER")
INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY")
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")

# 2. Upload sample data to blob container

### Connect to Azure Blob Storage

In [ ]:
from uuid import uuid4

from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient


# Get environment variables for Azure AI Vision
try:
    connection_string = BLOB_CONNECTION_STRING
    # container_name = os.getenv("BLOB_CONTAINER_NAME")
    container_name = BLOB_CONTAINER_NAME
except KeyError as e:
    print(f"Missing environment variable: {str(e)}")
    print("Set them before running this sample.")
    exit()

# Setup for Azure Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)


### Upload data/sample data to blob
Here we are uploading our data/samples images to **Azure Blob Storage** so they can be accessible by **Azure AI Search** for indexing

In [ ]:
local_folder_path = "../data/samples"

try:
    container_client.create_container()
except Exception as e:
    print(f"Container already exists: {e}")
    pass

# Upload files from the local folder to the blob container
for root, dirs, files in os.walk(local_folder_path):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
        
        with open(file_path, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)
            print(f"Uploaded {file_name} to {container_name}")

print("All files uploaded successfully.")

# 3. Create multimodal index in **Azure AI Search**

### Connect to Azure AI Serch

In [ ]:
# User-specified parameter
USE_AAD_FOR_SEARCH = False  # Set this to False to use API key for authentication

def authenticate_azure_search(api_key=None, use_aad_for_search=False):
    if use_aad_for_search:
        print("Using AAD for authentication.")
        credential = DefaultAzureCredential()
    else:
        print("Using API keys for authentication.")
        if api_key is None:
            raise ValueError("API key must be provided if not using AAD for authentication.")
        credential = AzureKeyCredential(api_key)
    return credential

azure_search_credential = authenticate_azure_search(api_key=AZURE_SEARCH_API_KEY, use_aad_for_search=USE_AAD_FOR_SEARCH)


### Create a blob data source connector on Azure AI Search

Datasource in Azure Ai search is pointing to a place where images are located. In our case it will be a blob storage where we upploaded our data earlier

In [ ]:
def create_or_update_data_source(indexer_client, container_name, connection_string, index_name):
    """
    Create or update a data source connection for Azure AI Search.
    """
    container = SearchIndexerDataContainer(name=container_name)
    data_source_connection = SearchIndexerDataSourceConnection(
        name=f"{index_name}-blob",
        type="azureblob",
        connection_string=connection_string,
        container=container
    )
    try:
        indexer_client.create_or_update_data_source_connection(data_source_connection)
        print(f"Data source '{index_name}-blob' created or updated successfully.")
    except Exception as e:
        raise Exception(f"Failed to create or update data source due to error: {e}")

# Create a SearchIndexerClient instance
indexer_client = SearchIndexerClient(AZURE_SEARCH_ENDPOINT, azure_search_credential)

# Call the function to create or update the data source
create_or_update_data_source(indexer_client, BLOB_CONTAINER_NAME, BLOB_CONNECTION_STRING, INDEX_NAME)

### Create a search index

Here we will define our multimodal search index and all necessery components. 
Our index will contain the following fields to search
- text field with image caption 
- vector fields with caption text vector
- vector field with image vector
- field with image path

In [ ]:
def create_fields():
    # Creates the fields for the search index based on the specified schema.
    return [
        SimpleField(
            name="id", type=SearchFieldDataType.String, key=True, filterable=True
        ),
        SearchField(name="caption", type=SearchFieldDataType.String, searchable=True), # image caption
        SearchField(name="metadata_storage_path", type=SearchFieldDataType.String, searchable=True), # image path
        SearchField(
            name="captionVector", # vectorized caption
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            vector_search_dimensions=1024,
            vector_search_profile_name="myHnswProfile",
            stored=False,
        ),
        SearchField(
            name="imageVector", # vectorized image
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            vector_search_dimensions=1024,
            vector_search_profile_name="myHnswProfile",
            stored=False,
        ),
    ]


def create_vector_search_configuration():
    # Creates the vector search configuration for the search index.
    # In the configuration we specify the algorithms, compressions, vectorizers to vectorise our queries and execute search

    return VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw",
                parameters=HnswParameters(
                    m=4,
                    ef_construction=400,
                    ef_search=500,
                    metric=VectorSearchAlgorithmMetric.COSINE,
                ),
            )
        ],
        compressions=[
            ScalarQuantizationCompressionConfiguration(
                name="myScalarQuantization",
                rerank_with_original_vectors=True,
                default_oversampling=10,
                parameters=ScalarQuantizationParameters(quantized_data_type="int8"),
            )
        ],
        vectorizers=[
            AIServicesVisionVectorizer(
                name="myAIServicesVectorizer",
                kind="aiServicesVision",
                ai_services_vision_parameters=AIServicesVisionParameters(
                    model_version="2023-04-15",
                    resource_uri=AZURE_AI_VISION_ENDPOINT,
                    api_key=AZURE_AI_VISION_API_KEY,
                ),
            )
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw",
                compression_configuration_name="myScalarQuantization",
                vectorizer="myAIServicesVectorizer",
            )
        ],
    )


# Creating an index with the specified fields and vector search configuration
def create_search_index(index_client, index_name, fields, vector_search):
    """Creates or updates a search index."""
    index = SearchIndex(
        name=index_name,
        fields=fields,
        vector_search=vector_search,
    )
    index_client.create_or_update_index(index=index)

# Create a SearchIndexClient instance for further operations
index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_ENDPOINT, credential=azure_search_credential
)
fields = create_fields()
vector_search = create_vector_search_configuration()

# Create the search index with the adjusted schema
create_search_index(index_client, INDEX_NAME, fields, vector_search)
print(f"Created index: {INDEX_NAME}")

### Create a Skillset   

Skillset is a collection of cognitive skills that are applied to your data during the indexing process. These skills can include a variety of AI-powered capabilities such as natural language processing, image analysis, and custom machine learning models. The purpose of a skillset is to enrich your data, extracting useful information and transforming it into a searchable format.

In our case we need skills for:
- creating an image caption from an image
- creating text embedding from the caption
- creating image embedding
- additional merge skill to flatten complex text data that may include many captions and tags



In [ ]:
def create_image_caption_skill(): # Create a skill to generate image caption
    return ImageAnalysisSkill(
        name="image-description-skill",
        description="Skill to generate caption for image",
        context="/document/normalized_images/*",
        inputs=[InputFieldMappingEntry(name="image", source="/document/normalized_images/0")],
        outputs=[OutputFieldMappingEntry(name="description", target_name="description")],
        defaultLanguageCode = "en",
        visualFeatures = ["Description"],
    )

def create_merge_skill(): # Create a skill to merge text and tags from image caption
    return MergeSkill(
        name="text-merge-skill",
        description="merge text and tags from image caption",
        context="/document/normalized_images/*",
        #inputs=[InputFieldMappingEntry(name="text", source="/document/normalized_images/*/description/captions/0/text"), InputFieldMappingEntry(name="itemsToInsert", source="/document/normalized_images/0/description/tags")],
        inputs=[InputFieldMappingEntry(name="itemsToInsert", source="/document/normalized_images/0/description/captions/*/text")], 
        outputs=[OutputFieldMappingEntry(name="mergedText", target_name="caption")],
        insertPreTag=", ",
        insertPostTag=""
    )

 

def create_text_embedding_skill(): # Create a skill to generate embeddings for text
    return VisionVectorizeSkill(
        name="text-embedding-skill",
        description="Skill to generate embeddings for text via Azure AI Vision",
        context="/document/normalized_images/*",
        model_version="2023-04-15",
        inputs=[InputFieldMappingEntry(name="text", source="/document/normalized_images/0/caption")],
        outputs=[OutputFieldMappingEntry(name="vector", target_name="captionVector")],
    )

def create_image_embedding_skill(): # Create a skill to generate embeddings for image
    return VisionVectorizeSkill(
        name="image-embedding-skill",
        description="Skill to generate embeddings for image via Azure AI Vision",
        context="/document/normalized_images/*",
        model_version="2023-04-15",
        inputs=[InputFieldMappingEntry(name="image", source="/document/normalized_images/*")],
        outputs=[OutputFieldMappingEntry(name="vector", target_name="imageVector")],
    )

# Create a skillset with the specified skills
def create_skillset(client, skillset_name, image_caption_skill, merge_skill, text_embedding_skill, image_embedding_skill):
    skillset = SearchIndexerSkillset(
        name=skillset_name,
        description="Skillset for generating embeddings",
        skills=[image_caption_skill, merge_skill, text_embedding_skill, image_embedding_skill],
        cognitive_services_account=CognitiveServicesAccountKey(
            key=AZURE_AI_VISION_API_KEY,
            description="AI Vision Multi Service Account",
        ),
    )
    client.create_or_update_skillset(skillset)


# Create a skillset with the specified skills
skillset_name = f"{INDEX_NAME}-skillset"
image_caption_skill = create_image_caption_skill()
text_embedding_skill = create_text_embedding_skill()
image_embedding_skill = create_image_embedding_skill()
merge_skill = create_merge_skill()

create_skillset(indexer_client, skillset_name, image_caption_skill, merge_skill, text_embedding_skill, image_embedding_skill)
print(f"Created skillset: {skillset_name}")

### Run Indexer

Now, when we have defined all index components
- index description 
- index configuration
- skills to enrich data

we can execute the Indexer and finally build a Search Index to look up images

In [ ]:
# Create an indexer that defins how to index the data and map the fields
def create_and_run_indexer(indexer_client, indexer_name, skillset_name, index_name, data_source_name):
    indexer = SearchIndexer(
        name=indexer_name,
        description="Indexer to index documents and generate embeddings",
        skillset_name=skillset_name,
        target_index_name=index_name,
        data_source_name=data_source_name,
        parameters=IndexingParameters(
            configuration=IndexingParametersConfiguration(
                #parsing_mode=BlobIndexerParsingMode.JSON_ARRAY,
                image_action=BlobIndexerImageAction.GENERATE_NORMALIZED_IMAGES,
                query_timeout=None,
            ),
        ),
        #field_mappings=[FieldMapping(source_field_name="id", target_field_name="id")],
        output_field_mappings=[
            FieldMapping(source_field_name="/document/normalized_images/*/caption", target_field_name="caption"),
            FieldMapping(source_field_name="/document/normalized_images/0/captionVector", target_field_name="captionVector"),
            FieldMapping(source_field_name="/document/normalized_images/0/imageVector", target_field_name="imageVector"),
            FieldMapping(source_field_name="/document/metadata_storage_path", target_field_name="metadata_storage_path"),
        ],
    )

    indexer_client.create_or_update_indexer(indexer)
    print(f"{indexer_name} created or updated.")

    indexer_client.run_indexer(indexer_name)
    print(f"{indexer_name} is running. If queries return no results, please wait a bit and try again.")


# Run the indexer
data_source_name = f"{INDEX_NAME}-blob"
indexer_name = f"{INDEX_NAME}-indexer"

create_and_run_indexer(indexer_client, indexer_name, skillset_name, INDEX_NAME, data_source_name)

Building an index will take several minutes. 
Wait a bit and move to exploring different search option for our multimodal index in [Lab 5.2](./5_2_azure_ai_search_multimodal.ipynb)